In [74]:
import bovespa
import pandas as pd
from datetime import datetime, timedelta
import re

In [55]:
acao = 'PETR4'
opcoes = 'PETRF1'
data_vencimento_opcoes = '2020/16/01'

acao = bovespa.get_values(acao)
print('Acao:\n')
display(acao)

Acao:



,Papel,Descrição,Data,Último,Neg,R$,Var
0,PETR4,PETROBRAS PN N2,06/05 17:59,17.24,80730,1.143.831K,"-3,9%"


In [56]:
# Separar ticker, descricao, data, valor
opcao = bovespa.get_values(opcoes, exact=False)
print('Opcao:\n')
display(opcao)

Opcao:



,Papel,Descrição,Data,Último,Neg,R$,Var
0,PETRF1,"PETR PN 26,30",06/05 16:36,0.06,2,0K,+0%
1,PETRF113,"PETR PN 11,30",06/05 16:00,6.36,3,0K,"-9,14%"
2,PETRF118,"PETR PN 11,80",06/05 16:01,5.92,6,0K,"-11,64%"
3,PETRF119,"PETRE /EJ PN 11,90",16/05 12:24,13.95,18,25K,"-1,34%"
4,PETRF123,"PETR PN 12,30",06/05 15:20,5.44,4,0K,"-12,26%"
5,PETRF133,"PETR PN 13,30",06/05 15:08,4.45,39,0K,-11%
6,PETRF135,"PETR PN 12,80",06/05 16:47,4.88,56,40K,"-13,78%"
7,PETRF138,"PETR PN 13,80",06/05 15:42,4.06,51,0K,"-17,48%"
8,PETRF140,"PETRE PN 14,05",06/05 16:48,3.85,122,0K,"-13,29%"
9,PETRF143,"PETR PN 14,30",06/05 16:47,3.62,46,0K,"-18,47%"


In [71]:
# Processa opção
def proc(acao, opcao, filter_type=True, filter_date=None):
    def find_type(description):
        if any('PN'==s for s in description):
            return('PN')
        elif any('ON'==s for s in description):
            return('ON')
        else:
            return('')

    ignored = False
    nome_acao = acao.iloc[0]['Papel']
    valor_acao = acao.iloc[0]['Último']
    descricao_acao = acao.iloc[0]['Descrição'].split()
    tipo_acao = find_type(descricao_acao)
    #descricao_acao[1]

    df = pd.DataFrame(columns=['acao', 'tipo acao', 'valor acao','opcao', 'tipo_opcao', 'valor opcao','strike', 'volume', 'lucro %'])
    for index, row in opcao.iterrows():
        nome_opcao = row['Papel']
        descricao_opcao = row['Descrição'].split()
        strike_opcao = float(descricao_opcao[-1].replace(',','.'))
        tipo_opcao = find_type(descricao_opcao)

        # Ignora opcoes de tipos diferentes (PN e ON)
        # TODO: Muito sujeito a dar pau isso aqui
        if tipo_acao != tipo_opcao:
            print('Ignorando opcao {} {} devido tipo diferente'.format(nome_opcao,' '.join(descricao_opcao)))
            continue

        # Descarta opcoes de dias antigos
        data_hora = row['Data']
        data = datetime.strptime(data_hora,  "%d/%m\xa0%H:%M")
        #now = datetime.now()
        now = filter_date
        if filter_date is not None:
            if data.month != now.month or data.day != now.day:
                print('Ignorando opcao {} {} devido data antiga'.format(nome_opcao,' '.join(descricao_opcao)))
                continue


        valor_opcao = row['Último']        
        volume_opcao = float(re.sub('\D','',row['R$']))
        if strike_opcao >= valor_acao:
            lucro = valor_opcao
        else:
            lucro = strike_opcao - valor_acao + valor_opcao
        p_lucro = (lucro / valor_acao) * 100

        #df = pd.DataFrame({'acao':[nome_acao], 'valor acao':[valor_acao], 'opcao':[nome_opcao], 'valor opcao':[valor_opcao],                    'strike':strike_opcao, 'lucro %':p_lucro})
        df.loc[index] = [nome_acao + ' ' + ' '.join(descricao_acao), tipo_acao, valor_acao, nome_opcao + ' ' + ' '.join(descricao_opcao), tipo_opcao, valor_opcao,strike_opcao, volume_opcao, p_lucro]
 
    return df

In [75]:
day = datetime.today() - timedelta(days=1)
teste = proc(acao, opcao, filter_date = day)
teste.sort_values('lucro %', ascending=False)

Ignorando opcao PETRF119 PETRE /EJ PN 11,90 foi ignorada devido data antiga
Ignorando opcao PETRF157 PETRE /EJ PN 14,90 foi ignorada devido data antiga
Ignorando opcao PETRF159 PETR /EJ PN 15,15 foi ignorada devido data antiga
Ignorando opcao PETRF171 PETRE /EJ PN 16,40 foi ignorada devido data antiga
Ignorando opcao PETRF18 PETR ON 18,00 devido tipo diferente
Ignorando opcao PETRF182 PETR /EJ PN 18,15 foi ignorada devido data antiga
Ignorando opcao PETRF19 PETR ON 19,00 devido tipo diferente
Ignorando opcao PETRF192 PETR /EJ PN 19,15 foi ignorada devido data antiga


,acao,tipo acao,valor acao,opcao,tipo_opcao,valor opcao,strike,volume,lucro %
24,PETR4 PETROBRAS PN N2,PN,17.24,"PETRF173 PETR PN 17,30",PN,1.52,17.30,294.0,8.816705
22,PETR4 PETROBRAS PN N2,PN,17.24,"PETRF170 PETRE PN 17,05",PN,1.65,17.05,222.0,8.468677
26,PETR4 PETROBRAS PN N2,PN,17.24,"PETRF178 PETR PN 17,80",PN,1.39,17.80,0.0,8.062645
25,PETR4 PETROBRAS PN N2,PN,17.24,"PETRF175 PETRE PN 17,55",PN,1.38,17.55,0.0,8.004640
21,PETR4 PETROBRAS PN N2,PN,17.24,"PETRF168 PETR PN 16,80",PN,1.79,16.80,0.0,7.830626
20,PETR4 PETROBRAS PN N2,PN,17.24,"PETRF165 PETRE PN 16,55",PN,1.97,16.55,0.0,7.424594
19,PETR4 PETROBRAS PN N2,PN,17.24,"PETRF163 PETR PN 16,30",PN,2.09,16.30,0.0,6.670534
28,PETR4 PETROBRAS PN N2,PN,17.24,"PETRF180 PETRE PN 18,05",PN,1.15,18.05,0.0,6.670534
18,PETR4 PETROBRAS PN N2,PN,17.24,"PETRF160 PETRE PN 16,05",PN,2.28,16.05,0.0,6.322506
30,PETR4 PETROBRAS PN N2,PN,17.24,"PETRF183 PETR PN 18,30",PN,1.06,18.30,0.0,6.148492
